# Modelo

La optimizacion la hago con el dataset undersampleado porque sino tarda una eternidad.

En el proximo script entreno ahi si con la totalidad de los datos.

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01_undersampled_10_24M_altbinaria.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [3]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105] # dejo afuera 202106 para test

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(376300, 679)

In [4]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria','foto_mes'], axis=1) # saco foto mes
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [6]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [ ]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 150), # segun statquest esto deberia ir de 8 a 32
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # eligir la clase
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    
    # Use callbacks for early stopping
    early_stopping_cb = lgb.early_stopping(stopping_rounds=50) # creo que en min_delta le tendria que pasar un parametro con la ganancia que considero irrelevante
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior (ahora le puso 100 para mostrarnos, pero hay que ponerle un numero alto, 10.000, 200.000)
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [ ]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia2_lgbm_v17" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-23 13:35:02,047] A new study created in RDB with name: competencia2_lgbm_v16


In [37]:
study.optimize(objective, n_trials=100)

Training until validation scores don't improve for 127 rounds
Early stopping, best iteration is:
[668]	cv_agg's valid gan_eval: 6.57895e+08 + 2.63082e+06


[I 2024-11-23 13:37:09,064] Trial 0 finished with value: 3289475000.0 and parameters: {'num_leaves': 11, 'learning_rate': 0.06467374047757601, 'min_data_in_leaf': 534, 'feature_fraction': 0.8713268509843334, 'bagging_fraction': 0.7201376276754294}. Best is trial 0 with value: 3289475000.0.


Training until validation scores don't improve for 143 rounds
Early stopping, best iteration is:
[614]	cv_agg's valid gan_eval: 6.59893e+08 + 3.25673e+06


[I 2024-11-23 13:40:30,838] Trial 1 finished with value: 3299464000.0 and parameters: {'num_leaves': 31, 'learning_rate': 0.05369042042420028, 'min_data_in_leaf': 330, 'feature_fraction': 0.2733866757518019, 'bagging_fraction': 0.5539036296548145}. Best is trial 1 with value: 3299464000.0.


Training until validation scores don't improve for 172 rounds
Early stopping, best iteration is:
[545]	cv_agg's valid gan_eval: 6.61641e+08 + 2.52609e+06


...

[I 2024-11-24 01:46:37,797] Trial 97 finished with value: 3318455000.0 and parameters: {'num_leaves': 119, 'learning_rate': 0.02272741057992425, 'min_data_in_leaf': 1671, 'feature_fraction': 0.24044504228503594, 'bagging_fraction': 0.8563911396611095}. Best is trial 91 with value: 3323313000.0.


Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[783]	cv_agg's valid gan_eval: 6.63328e+08 + 3.50961e+06


[I 2024-11-24 01:51:54,880] Trial 98 finished with value: 3316642000.0 and parameters: {'num_leaves': 111, 'learning_rate': 0.01994768875221883, 'min_data_in_leaf': 1402, 'feature_fraction': 0.32425653509277574, 'bagging_fraction': 0.8727722029812267}. Best is trial 91 with value: 3323313000.0.


Training until validation scores don't improve for 556 rounds
Early stopping, best iteration is:
[1276]	cv_agg's valid gan_eval: 6.62609e+08 + 2.99066e+06


[I 2024-11-24 02:08:15,807] Trial 99 finished with value: 3313044000.0 and parameters: {'num_leaves': 137, 'learning_rate': 0.00987489383128112, 'min_data_in_leaf': 1612, 'feature_fraction': 0.6674737750850779, 'bagging_fraction': 0.8312494394617136}. Best is trial 91 with value: 3323313000.0.


Analizamos los resultados as usual

In [38]:
optuna.visualization.plot_optimization_history(study)

In [39]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [40]:
plot_slice(study)

In [41]:
plot_contour(study)

In [42]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [43]:
study.best_trial.params

{'num_leaves': 102,
 'learning_rate': 0.01243700594135768,
 'min_data_in_leaf': 1647,
 'feature_fraction': 0.26739987658099496,
 'bagging_fraction': 0.8602545673067901}

In [44]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

1026